In [5]:
import os
import re
from elftools.elf.elffile import ELFFile
import Stm32ImageAddHeader

In [31]:
def add_header(elf_file, bin_file, out_file, text_name=".text"):
    if os.path.isfile(elf_file) is False:
        raise Exception("No such file:{}".format(elf_file))

    if os.path.isfile(bin_file) is False:
        raise Exception("No such file:{}".format(bin_file))

    with open(elf_file, "rb") as file:
        elf_file_h = ELFFile(file)
        text_section = elf_file_h.get_section_by_name(text_name)

        load_address = text_section.header["sh_addr"]
        entry_point = elf_file_h.header.e_entry

    stm32im = Stm32ImageAddHeader.Stm32Image(
        header_major_ver = 0x02,
        header_minor_ver = 0x00,
        entry            = entry_point,
        loadaddr         = load_address,
    )
    ret = stm32im.generate(bin_file, out_file)
    if ret != 0:
      print("Aborted")
      return ret

    stm32im.print_header()
    print("%s generated" % out_file)

The entire program in the following example fits in SYSRAM and does not have a separate `.text` section. Instead, everything exectutable is placed in the `.RESET` section by the linker file `stm32mp13xx_a7_sysram.ld`.

Unlike MP15x, MP23x, MP25x lines, the STM32MP135x does not need the "Binary type" field in the STM32 header. Therefore we set it to 0.

In [34]:
add_header(
    elf_file  = "blink_cubeide.elf",
    bin_file  = "blink_cubeide.bin",
    out_file  = "blink.stm32",
    text_name = ".RESET",
)

| Magic number            | @0x00   0 | sz   4 | 53 54 4D 32  
| Image Signature         | @0x04   4 | sz  64 | 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 ... 
| Image Checksum          | @0x44  68 | sz   4 | EA 3B 65 00  
| Header version          | @0x48  72 | sz   4 | 00 00 02 00  
| Image Length            | @0x4c  76 | sz   4 | E4 F6 00 00  
| Image Entry  Point      | @0x50  80 | sz   4 | 00 00 FE 2F  
| Reserved1               | @0x54  84 | sz   4 | 00 00 00 00  
| Load address            | @0x58  88 | sz   4 | 00 00 FE 2F  
| Reserved2               | @0x5c  92 | sz   4 | 00 00 00 00  
| Version Number          | @0x60  96 | sz   4 | 00 00 00 00  
| Extension flags         | @0x64 100 | sz   4 | 00 00 00 80  
| Post headers length     | @0x68 104 | sz   4 | 80 01 00 00  
| Binary type             | @0x6c 108 | sz   4 | 00 00 00 00  
| PAD                     | @0x70 112 | sz  16 | 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  
| Extension header type   | @0x80 128 | sz 